In [59]:
#-*- coding:utf-8 -*-
import json
import requests
from urllib.parse import quote

class Searcher(object):
#     _host = '10.31.141.51:8130'
    _host = '123.57.173.102:8130'
    _timeout_s = 3

    @classmethod
    def question_search(cls, query, size=200):
        url = ('http://%s/in/nlp/tob/datasearch?size=%s&score=bow&condition=5<50%%&query=%s'
            % (cls._host, size, query))
#         url = 'http://10.31.141.51:8130/in/nlp/tob/datasearch?size=100&score=wmd&query='';
        r = requests.get(url)
#         print (r.content.decode('unicode_escape'))
        rsp = r.json()
        sim_questions = []
        score = []
        if rsp['errno'] == 0:
            result = [{
                'question':x['info']['question'],
                'score':x['score']
            } for x in rsp['results']['results'] if x['score']>0.58]
#             if x['score']>0.75
#             sim_questions = [x['info']['question'] for x in rsp['results']['results']]
#             score = [x['score'] for x in rsp['results']['results']]
            #print(score)
        return result


def print_sim_questions(sim_questions):
    print (len(sim_questions))
    for sim_ques in sim_questions:
        print( sim_ques)
        
        
def write__sim_questions(question,sim_questions,score,file):
    file.write(question.strip('\t')+' ')
    file.write(' '.join(sim_questions)+'\n')

# import random
# num = random.sample(range(1,190),50)
# with open('log/msg_neg.txt','r') as f_neg,open('log/msg_pos.txt','r') as f_pos,open('log/data.txt','w') as f_data:
#     line_pos = [x for x in f_pos]
#     line_neg = [x for x in f_neg]
#     for i in num:
#         f_data.write(line_pos[i]+line_neg[i])

            


In [60]:
if '__main__' == __name__:
#     result = Searcher.question_search('小来你真棒')
#     print_sim_questions(result)
    with open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_pos_seed.txt','r') as fr_pos,\
         open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_pos_es.txt','w') as fw_pos,\
         open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_neg_seed.txt','r') as fr_neg, \
         open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_neg_es.txt','w') as fw_neg :
#         open('log/data.txt','r')as fr,open('log/data_recall.txt','w') as fw:
       
#         for msg in fr_pos:
#             results = Searcher.question_search(msg)
# #             print(results)
#             fw_pos.write('%s%s\n' % (
#                         msg.strip()+'\n',
#                         '\n'.join('%s' % x['question'].strip() for x in results)
#                         )
#                         )
        for msg in fr_neg:
            results = Searcher.question_search(msg)
            fw_neg.write('%s%s\n' % (
                        msg.strip()+'\n',
                        '\n'.join('%s' % x['question'].strip() for x in results)
                        )
                        )
            
#              fw_pos.write('%s\t%s\n' % (
#                     msg.strip('\n'),
#                     '\t'.join(['%.8f|%s'%(x['score'],x['question']) for x in results])
#                 )
#             )
        
   

In [50]:
#产生随机数
import random 
def generate_num(length):
    num = range(length)
    return random.sample(num,200)

In [51]:
#抽样es检索并去重的数据 各100条

with open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_neg_es_uniq', 'r') as fn,\
     open('/home/works/data/xuman/sentiment_analysis/allmsg/msg_pos_es_uniq','r') as fp,\
     open ('/home/works/data/xuman/sentiment_analysis/allmsg/random_pos.txt','w') as frand_p,\
     open ('/home/works/data/xuman/sentiment_analysis/allmsg/random_neg.txt','w') as frand_n:
    line_neg = [line for line in fn]
    line_pos = [line for line in fp]
    len_p = len(line_pos)
    len_n = len(line_neg)
    for i in generate_num(len_n):
#         print(line_neg[i])
        frand_n.write(line_neg[i])
    for j in generate_num(len_p):
        frand_p.write(line_pos[j])
    
# frand.close()
    
    
    

In [ ]:
if '__main__' == __name__:
    with open('/home/works/data/xuman/sentiment_analysis/allmsg/random.txt','r') as f,\
        open('/home/works/data/xuman/sentiment_analysis/allmsg/random_es.txt','w') as fw:
        for msg in f:
            results = Searcher.question_search(msg)
#             print(len(results))
            fw.write('%s\t%s\n' % (
                    msg.strip('\n'),
                    '\t'.join(['%.8f|%s'%(x['score'],x['question']) for x in results])
                )
            )      
   

In [ ]:
# 召回率
if '__main__' == __name__:
#     with open('log/msg_pos.txt','r') as fr_pos, open('log/es_msg_pos.txt','w') as fw_pos,open('log/msg_neg.txt','r') as fr_neg, open('log/es_msg_neg.txt','w') as fw_neg :
    with open('/home/works/data/xuman/sentiment_analysis/allmsg/recall.csv','r')as fr,open('/home/works/data/xuman/sentiment_analysis/allmsg/recall_2.txt','w') as fw:
     
        for msg in fr:
            results = Searcher.question_search(msg)
#             print(results)
            fw.write('%s\t%s\n' % (
                    msg.strip('\n'),
                    '\t'.join(['%.8f|%s'%(x['score'],x['question']) for x in results])
                )
            )



In [ ]:
# calculate recall
import pandas as pd
df = pd.read_csv('/home/works/data/xuman/sentiment_analysis/allmsg/recall.csv',header = None)
df.drop(df.index[201:], axis=0, inplace=True)
df.drop(df.columns[7:], axis=1, inplace=True)
df

In [ ]:
num_1 = 0
num_0 = 0.0
type(num_1)
recall = []
for i in range(1,len(df)+1):
    if df.iloc[i-1,6] == '1':
        num_1 += 1
    recall.append(num_1/i)
df['recall'] =recall 
df.to_csv('/home/works/data/xuman/sentiment_analysis/allmsg/recall_2.csv',index=False,sep=',')

In [ ]:
df.head(200)

In [14]:
176/200

0.88

In [23]:
155/200

0.775

In [24]:
6124/1600

3.8275

In [41]:
168/200

0.84

In [63]:
3913/1021

3.832517140058766

In [61]:
3716/1600

2.3225

In [62]:
184/200

0.92